In [ ]:
!pip install datasets transformers torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.5 MB/s eta 0:00:00


In [ ]:
import datasets
import pandas as pd
import numpy as np
import torch
import random
from tqdm import tqdm

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Load the CNN/DailyMail dataset
cnn_dataset = datasets.load_dataset("abisee/cnn_dailymail", "3.0.0")

# Convert to pandas DataFrame for easier manipulation
train_df = pd.DataFrame(cnn_dataset["train"])
val_df = pd.DataFrame(cnn_dataset["validation"])
test_df = pd.DataFrame(cnn_dataset["test"])

# Sample 1% of the training data
sample_size = int(len(train_df) * 0.001)
train_sample = train_df.sample(n=sample_size, random_state=42)

print(f"Full training set size: {len(train_df)}")
print(f"1% sample size: {len(train_sample)}")


Full training set size: 287113
1% sample size: 287


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def tokenize_document(document):
    # Split document into sentences
    sentences = sent_tokenize(document)

    # Preprocess each sentence
    processed_sentences = [preprocess_text(sentence) for sentence in sentences]

    # Remove empty sentences
    processed_sentences = [s for s in processed_sentences if s.strip()]

    return processed_sentences

# Apply preprocessing to the sampled data
train_sample['processed_article'] = train_sample['article'].apply(tokenize_document)
train_sample['processed_highlights'] = train_sample['highlights'].apply(tokenize_document)

# Create labels for extractive summarization (1 if sentence is in highlights, 0 otherwise)
def create_extractive_labels(article_sentences, highlight_sentences):
    labels = []
    for sentence in article_sentences:
        # Check if this sentence is similar to any highlight sentence
        is_in_highlights = any(
            similarity_score(sentence, highlight) > 0.7
            for highlight in highlight_sentences
        )
        labels.append(1 if is_in_highlights else 0)
    return labels

def similarity_score(sent1, sent2):
    # Simple word overlap similarity
    words1 = set(word_tokenize(sent1))
    words2 = set(word_tokenize(sent2))

    if not words1 or not words2:
        return 0

    overlap = len(words1.intersection(words2))
    return overlap / max(len(words1), len(words2))

# Create extractive labels
train_sample['extractive_labels'] = [
    create_extractive_labels(article, highlight)
    for article, highlight in zip(train_sample['processed_article'], train_sample['processed_highlights'])
]


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel

class TextGraph:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.tfidf = TfidfVectorizer()

    def build_graph(self, document_sentences):
        # Extract all unique words
        all_words = set()
        for sentence in document_sentences:
            words = [word for word in word_tokenize(sentence) if word not in stopwords.words('english')]
            all_words.update(words)

        # Create node feature vectors
        sentence_features = self._encode_sentences(document_sentences)
        word_features = self._encode_words(list(all_words))

        # Calculate TF-IDF for edge weights
        self.tfidf.fit(document_sentences)
        tfidf_matrix = self.tfidf.transform(document_sentences)

        # Create edges (word-sentence connections)
        edges = []
        edge_weights = []

        for word_idx, word in enumerate(all_words):
            for sent_idx, sentence in enumerate(document_sentences):
                if word in word_tokenize(sentence):
                    # Word to sentence edge
                    edges.append([word_idx, sent_idx + len(all_words)])

                    # Get TF-IDF weight
                    word_id = self.tfidf.vocabulary_.get(word, -1)
                    weight = tfidf_matrix[sent_idx, word_id] if word_id != -1 else 0
                    edge_weights.append(weight)

        return {
            'sentence_features': sentence_features,
            'word_features': word_features,
            'edges': torch.tensor(edges).t().contiguous() if edges else torch.zeros((2, 0), dtype=torch.long),
            'edge_weights': torch.tensor(edge_weights) if edge_weights else torch.zeros(0),
            'sentences': document_sentences,
            'num_sentences': len(document_sentences),
            'num_words': len(all_words)
        }

    def _encode_sentences(self, sentences):
        # Encode sentences using BERT
        with torch.no_grad():
            encoded_sentences = []
            for sentence in sentences:
                inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=128)
                outputs = self.bert(**inputs)
                # Use CLS token as sentence representation
                encoded_sentences.append(outputs.last_hidden_state[:, 0, :].squeeze())

            if encoded_sentences:
                return torch.stack(encoded_sentences)
            else:
                return torch.zeros((0, 768))

    def _encode_words(self, words):
        # Encode words using BERT
        with torch.no_grad():
            encoded_words = []
            for word in words:
                inputs = self.tokenizer(word, return_tensors='pt')
                outputs = self.bert(**inputs)
                # Use mean of token embeddings as word representation
                encoded_words.append(outputs.last_hidden_state.mean(dim=1).squeeze())

            if encoded_words:
                return torch.stack(encoded_words)
            else:
                return torch.zeros((0, 768))


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class SummarizationGNN(nn.Module):
    def __init__(self, word_dim=768, sentence_dim=768, hidden_dim=256, num_heads=8):
        super(SummarizationGNN, self).__init__()

        # Word and sentence projections to the same dimension
        self.word_projection = nn.Linear(word_dim, hidden_dim)
        self.sentence_projection = nn.Linear(sentence_dim, hidden_dim)

        # GAT layers
        self.word_to_sentence = GATConv(hidden_dim, hidden_dim, heads=num_heads, concat=False)

        # Feed-forward networks after GAT
        self.sentence_ffn = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim)
        )

        # Sentence classifier
        self.sentence_classifier = nn.Linear(hidden_dim, 1)

    def forward(self, graph):
        # Get features and structure
        word_features = graph['word_features']
        sentence_features = graph['sentence_features']
        edges = graph['edges']
        edge_weights = graph['edge_weights']
        num_words = graph['num_words']

        # Handle empty graphs
        if word_features.size(0) == 0 or sentence_features.size(0) == 0 or edges.size(1) == 0:
            return torch.zeros(sentence_features.size(0))

        # Project features to the same dimension
        word_features = self.word_projection(word_features)
        sentence_features = self.sentence_projection(sentence_features)

        # Combine features
        all_features = torch.cat([word_features, sentence_features], dim=0)

        # Word to sentence message passing
        updated_features = self.word_to_sentence(all_features, edges, edge_weights)

        # Apply FFN to sentence features
        sentence_updated = self.sentence_ffn(updated_features[num_words:])

        # Sentence classification
        sentence_scores = self.sentence_classifier(sentence_updated).squeeze(-1)

        return sentence_scores


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class GraphDataset(Dataset):
    def __init__(self, dataframe, text_graph):
        self.dataframe = dataframe
        self.text_graph = text_graph

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        article_sentences = self.dataframe.iloc[idx]['processed_article']
        labels = self.dataframe.iloc[idx]['extractive_labels']

        # Build graph
        graph = self.text_graph.build_graph(article_sentences)

        # Convert labels to tensor and move to device
        graph['labels'] = torch.tensor(labels, dtype=torch.float).to(device)

        return graph

def collate_graphs(batch):
    return batch

# Initialize text graph builder
text_graph = TextGraph()

# Create dataset
train_dataset = GraphDataset(train_sample, text_graph)

# Create dataloader
train_loader = DataLoader(
    train_dataset,
    batch_size=1,  # Process one document at a time
    shuffle=True,
    collate_fn=collate_graphs
)

# Initialize model and move it to GPU
model = SummarizationGNN().to(device)

# Define loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 3



for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        graph = batch[0]  # Get the single graph from the batch

        # Skip empty graphs
        if graph['sentence_features'].size(0) == 0 or graph['word_features'].size(0) == 0 or graph['edges'].size(1) == 0:
            continue

        # Move all tensors to GPU
        graph['sentence_features'] = graph['sentence_features'].to(device)
        graph['word_features'] = graph['word_features'].to(device)
        graph['edges'] = graph['edges'].to(device)
        if 'edge_weights' in graph:
            graph['edge_weights'] = graph['edge_weights'].to(device)  # Move edge weights to GPU
        graph['labels'] = graph['labels'].to(device)

        # Forward pass
        scores = model(graph)

        # Calculate loss
        loss = criterion(scores, graph['labels'])

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Save the model
torch.save(model.state_dict(), "gnn_summarizer_model.pt")


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch 1/3, Loss: 0.0856


Epoch 2/3: 100%|██████████| 287/287 [49:37<00:00, 10.37s/it] 


Epoch 2/3, Loss: 0.0654


Epoch 3/3: 100%|██████████| 287/287 [49:34<00:00, 10.36s/it]

Epoch 3/3, Loss: 0.0817


In [ ]:

def generate_summary(article, model, text_graph, ratio=0.3):
    device = next(model.parameters()).device  # Get model's device

    # Preprocess article
    sentences = tokenize_document(article)

    # Build graph
    graph = text_graph.build_graph(sentences)

    # Skip empty graphs
    if graph['sentence_features'].size(0) == 0 or graph['word_features'].size(0) == 0 or graph['edges'].size(1) == 0:
        return ""  # Return empty summary if the graph is empty

    # Move all tensors in graph to the model's device
    graph['sentence_features'] = graph['sentence_features'].to(device)
    graph['word_features'] = graph['word_features'].to(device)
    graph['edges'] = graph['edges'].to(device)
    if 'edge_weights' in graph:
        graph['edge_weights'] = graph['edge_weights'].to(device)

    # Get sentence scores
    model.eval()
    with torch.no_grad():
        sentence_scores = model(graph)

    # Select top sentences
    num_sentences = graph['num_sentences']
    num_to_select = max(1, int(num_sentences * ratio))

    # Get indices of top sentences
    _, indices = torch.topk(sentence_scores, min(num_to_select, len(sentence_scores)))
    selected_indices = sorted(indices.tolist())

    # Generate summary
    original_sentences = sent_tokenize(article)
    summary_sentences = [original_sentences[i] for i in selected_indices if i < len(original_sentences)]
    summary = ' '.join(summary_sentences)

    return summary

# Test the model on a sample article
sample_article = test_df.iloc[0]['article']
generated_summary = generate_summary(sample_article, model, text_graph)
actual_summary = test_df.iloc[0]['highlights']

print("Generated Summary:")
print(generated_summary)
print("\nActual Summary:")
print(actual_summary)


Generated Summary:
The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. Rights group Human Rights Watch welcomed the development. The United States also said it "strongly" disagreed with the court's decision. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crim

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7c345d4869e35035f4c2881f1964c62bcfb4cb4a01017d2b4250b7651123b383
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def evaluate_summaries(generated_summaries, reference_summaries):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for gen_sum, ref_sum in zip(generated_summaries, reference_summaries):
        score = scorer.score(ref_sum, gen_sum)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    # Compute average scores
    avg_scores = {key: sum(values) / len(values) if values else 0.0 for key, values in scores.items()}
    return avg_scores

# Generate summaries for a small test set
test_sample = test_df.head(10)
generated_summaries = []
reference_summaries = []

for _, row in test_sample.iterrows():
    generated_summary = generate_summary(row['article'], model, text_graph)
    generated_summaries.append(generated_summary if generated_summary else " ")  # Avoid empty strings
    reference_summaries.append(row['highlights'])

# Calculate ROUGE scores
rouge_scores = evaluate_summaries(generated_summaries, reference_summaries)

# Display ROUGE scores
print("ROUGE Scores:")
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")


ROUGE Scores:
ROUGE-1: 0.2843
ROUGE-2: 0.1019
ROUGE-L: 0.1971


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
# Load tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split
# Generate GNN-based summaries
train_sample['gnn_summary'] = train_sample['article'].apply(lambda x: generate_summary(x, model, text_graph))

# Split into train and validation sets (90% train, 10% validation)
train_df, val_df = train_test_split(train_sample, test_size=0.1, random_state=42)



In [ ]:
from datasets import Dataset,DatasetDict

# Convert Pandas DataFrames to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_dict({
        "input_text": train_df["gnn_summary"].tolist(),
        "target_text": train_df["highlights"].tolist(),
    }),
    "validation": Dataset.from_dict({
        "input_text": val_df["gnn_summary"].tolist(),
        "target_text": val_df["highlights"].tolist(),
    })
})

# Extract train and validation datasets
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [ ]:
# Define tokenization function
def tokenize_function(batch):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=128)

    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Load pre-trained BART model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Map:   0%|          | 0/258 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")
wandb.login(key=secret_value_0)
wandb.init(project="Transformer + GNN")
# Define training arguments
training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    evaluation_strategy="epoch",  # Enables evaluation every epoch
    save_strategy="epoch",        # Saves model every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=0,  # Limits saved checkpoints to avoid storage issues
    load_best_model_at_end=True,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,  # Include validation set
)

# Train the model
trainer.train()

wandb: Currently logged in as: b22cs093 (b22cs093-prom-iit-rajasthan). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.274772
2,No log,1.225530
3,No log,1.262935


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=195, training_loss=1.3796649639423078, metrics={'train_runtime': 172.6718, 'train_samples_per_second': 4.482, 'train_steps_per_second': 1.129, 'total_flos': 838669481017344.0, 'train_loss': 1.3796649639423078, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("bart_finetuned")


In [ ]:
from rouge_score import rouge_scorer

# Ensure model is in evaluation mode
model.eval()

# ROUGE Evaluation Function
def evaluate_summaries(generated_summaries, reference_summaries):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for gen_sum, ref_sum in zip(generated_summaries, reference_summaries):
        score = scorer.score(ref_sum, gen_sum)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    avg_scores = {key: sum(values) / len(values) if values else 0.0 for key, values in scores.items()}
    return avg_scores

# Generate summaries for a small test set
test_sample = val_df.head(10)  # Using validation set
generated_summaries = []
reference_summaries = []

with torch.no_grad():  # Disable gradient tracking for inference
    for _, row in test_sample.iterrows():
        input_text = row['gnn_summary']

        if not input_text or not isinstance(input_text, str):  # Handle empty/missing summaries
            generated_summaries.append(" ")
            reference_summaries.append(row['highlights'])
            continue

        input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids.to(model.device)
        output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        generated_summaries.append(generated_summary)
        reference_summaries.append(row['highlights'])

# Calculate ROUGE scores
rouge_scores = evaluate_summaries(generated_summaries, reference_summaries)

# Display ROUGE scores
print("\nROUGE Scores (Fine-tuned BART on GNN Summaries):")
print(f"ROUGE-1: {rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_scores['rougeL']:.4f}")



ROUGE Scores (Fine-tuned BART on GNN Summaries):
ROUGE-1: 0.4520
ROUGE-2: 0.2314
ROUGE-L: 0.3117
